In [1]:
#%pip install scikit-learn
#%pip install nltk
#%pip install utils
#%pip install emoji

In [20]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
#from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player, remove_stopwords_from_text
import emoji

#INFO included in utils - but utils doesn't work for me here
def map_emoji_to_description(emoji_text): 
    emoji_description = emoji.demojize(emoji_text)
    return emoji_description
def remove_stopwords_from_text(text, stopwords_list_per_language):
    return remove_stopwords(text, stopwords=stopwords_list_per_language)

# Load Data

In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [27]:
# Filter out the Spanish data and reindex
df_en = df[df["language"] == "en"]

#Reset index
df_en = df_en.reset_index(drop=True)
df_en

0      {'content': 'Football\nFlorian Wirtz\'s goal f...
1      {'content': '[1/4]\xa0Soccer Football - Europa...
2      {'content': "By Will Pickworth For Mailonline\...
3      {'content': '\nBUENOS AIRES (AP) — World Cup w...
4      {'content': 'Sign In\nSign In\nThe Star Editio...
                             ...                        
297    {'content': 'Please refresh the page or naviga...
298    {'content': "By Oliver Salt For Mailonline\nPu...
299    {'content': 'We use cookies and other tracking...
300    {'content': 'Man City’s Alex Robertson makes d...
301    {'content': "\nLast updated on 19 March 202319...
Name: data, Length: 302, dtype: object

In [5]:
print(df_en.loc[5, 'data'])

{'content': "World Cup winners Argentina will celebrate with their fans at two home games in March against Panama and Curacao in their first matches since their triumph in Qatar.\nArgentina coach Lionel Scaloni announced a 35-man squad on Friday which included Lionel Messi, Angel Di Maria and the other members of Argentina's winning squad in Qatar.\nManchester United forward Alejandro Garnacho is rewarded for impressing at Old Trafford with his first-ever callup. The 18-year-old, in his breakthrough season with Erik ten Hag's side, has contributed five goals and six assists in 30 matches this campaign.\nIn their first games since beating France in the World Cup final, Argentina will play Panama on March 23 at the Monumental Stadium in Buenos Aires. Five days later, they will host Curacao in another friendly at the northern city of Santiago del Estero.\nScaloni, 44, earlier this week extended his contract until the end of the 2026 World Cup.\nSquad:\nGoalkeepers: Emiliano Martinez (Asto

# Remove duplicates

In [6]:
# Remove the similiar rows (The Function is imported from utils on top)

# Returned df is not working well

# Remove patterns

In [ ]:
#testing to find afftected rows
data_affected_row = df_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('new to', case=False)]

# Display the filtered rows
print(filtered_rows['data'].values)

["{'content': 'Please refresh the page or navigate to another page on the site to be automatically logged inPlease refresh your browser to be logged in\\nFind your bookmarks in your Independent Premium section, under my profile\\nFranck Kessie’s stoppage-time winner saw Barcelona sink Real Madrid in El Clasico and move 12 points clear at the top of LaLiga with 12 games to play.\\nBarca hit back after Ronald Araujo’s early own goal to level through Sergi Roberto on the stroke of half-time and set up the dramatic finale at the Nou Camp.\\nKessie pounced to finish off a cross from Alejandro Balde for his first Barcelona goal and help his side take a big step towards another domestic title.\\nBorja Iglesias scored the only goal as Real Betis bounced back from their midweek European defeat to Manchester United with a 1-0 win over Mallorca.\\nTwo late goals from Jose Luis Morales eased Villarreal to a 3-0 win at Osasuna, while Real Sociedad ended their run of four without a win and consolida

In [ ]:
# Create a copy
data_wo_pattern_en = df_en.copy()

#translate emojis into words
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group()), str(x)))

#remove patterns
patternlist_en = [
    r"^{\'content\': \'",
    r"^{\'content\'",
    r'http\S+',
    r"Copyright.*",
    r"Photo.*",
    r"All rights reserved.*",
    r'Published.*2023',
    r'Published by.*',
    r'Updated.*2023',
    r'Last updated.*2023',
    r'When you purchase through.*',
    r'New to.*',
    r'Please refresh.*',
    r'From the section.*',
    r'We use cookies.*',
    r'Sign In\nSign In\nThe Star Edition\nCHANGE LOCATION\nThis copy is for your personal non-commercial use only. To order\n    presentation-ready copies of Toronto Star content for distribution\n    to colleagues, clients or customers, or inquire about\n    permissions/licensing, please go to: www.TorontoStarReprints.com\n'
]
#do we want to keep authors of the text? #data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'By.*\n', "", str(x)))
#second version #data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'Edited by.*\n', "", str(x)))


In [ ]:
type(df_en)

pandas.core.frame.DataFrame

In [ ]:
data_wo_pattern_en2 = df_en.copy()

def remove_patterns_from_text(text, patterns):
    for pattern in patterns:
        text = re.sub(pattern, "", text)
    return text

# Apply remove_patterns_from_text function to 'data' column
data_wo_pattern_en2['data'] = data_wo_pattern_en2['data'].apply(lambda x: remove_patterns_from_text(x['content'], patternlist_en))

print(data_wo_pattern_en2['data'])

TypeError: string indices must be integers

In [ ]:
for pattern in patternlist_en:
    data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))
print(data_wo_pattern_en.loc[2, 'data'])
#it takes too long,if I would extend the pattern with + r'.*?\n' . But at the moment it deletes too much

: "By Will Pickworth For Mailonline\n\n\n5\nView  comments\n\nBayer Leverkusen, Shakhtar Donetsk, Union Berlin, Sevilla and Sporting Lisbon all joined Juventus,Manchester United\xa0and Roma\xa0in reaching the Europa League last-16.\nIt was a goal-heavy Thursday in the competition with 24 goals scored across the eight ties.\nShakhtar Donetsk continued their miraculous season as they overcame Rennes in the most dramatic fashion, beating them 5-4 on penalties.\nHaving led 2-1 from last week's first leg in Warsaw, a Karl Toko Ekambi strike gave the French side a 1-0 lead early on in the second half.\nThere were no more goals in the rest of normal time and the game headed into extra time with the score 2-2 on aggregate.\nShakhtar Donetsk reached the Europa League last-16 in miraculous fashion against Rennes\nKeeper Anatolii Trubin made three spectacular saves in the shootout to send them through\nBayer Leverkusen also reached the last-16 after beating AS Monaco 5-3 on penalties\nIbrahim Sal

In [ ]:
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('Erik ten Hag is reportedly looking', case=False)]

# Display the filtered rows
print(filtered_rows)

Empty DataFrame
Columns: [data, player, language, publishedAt]
Index: []


In [ ]:
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"Sign InnSign Inn*", "", str(x)))
print(data_wo_pattern_en.loc[5:15, 'data'])


5     : "World Cup winners Argentina will celebrate ...
6     Bayer Leverkusen became the third side to beat...
7     : "By Reuters\n\n\n7\nView  comments\n\nBayer ...
8     Bayer Leverkusen\'s Exequiel Palacios scored t...
9     by Tamojit Chakraborty March 19, 2023\nExequie...
10                                               : "\n.
11                                                     
12    : "Players from Barcelona and Real Madrid clas...
13    Craig Burley thinks Fulham players losing thei...
14    Argentina\'s World Cup winner Exequiel Palacio...
15    \nBarcelona took a huge step towards winning t...
Name: data, dtype: object


# Remove Stopwords

In [ ]:
# Words to keep
words_to_keep = {'don', 'didn', 'doesn', 'shouldn', 'couldn', 'wouldn', 'never', 'isn', 'cannot', 'except', 'no', 'neither', 'nor', 'cant', 'top', 'least'}

# Create modified stopwords list
modified_en_stopwords = STOPWORDS - words_to_keep

print(modified_en_stopwords)

frozenset({'must', 'by', 'although', 'almost', 'few', 'over', 'thereafter', 'moreover', 'under', 'former', 'becomes', 'he', 'at', 'well', 'give', 'latterly', 'thin', 'mill', 'yourself', 'towards', 'against', 'per', 'take', 'via', 'therefore', 'together', 'thus', 'etc', 'elsewhere', 'whereupon', 'sixty', 'hence', 'every', 'sincere', 'our', 'too', 'others', 'until', 'off', 'hereby', 'kg', 'alone', 'their', 'can', 'due', 'made', 'everywhere', 'where', 'hasnt', 'twelve', 'really', 'otherwise', 'see', 'any', 'afterwards', 'if', 'move', 'whether', 'namely', 'themselves', 'each', 'since', 'to', 'whereby', 'call', 'perhaps', 'thence', 'fifty', 'upon', 'i', 'now', 'whoever', 'bill', 'own', 'before', 'do', 'somehow', 'most', 'it', 'anyway', 'beforehand', 'have', 'me', 'during', 'an', 'detail', 'throughout', 'whose', 'does', 'amongst', 'thereby', 'un', 'is', 'them', 'other', 'than', 'about', 'another', 'very', 'noone', 'either', 'herself', 'eight', 'even', 'ltd', 'used', 'thru', 'across', 'as', '

In [ ]:
data_rm_en = data_wo_pattern_en.copy()

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_en['data'] = data_rm_en['data'].apply(lambda x: remove_stopwords_from_text(x, modified_en_stopwords))

# Remove noise

In [ ]:
# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_en['data'] = data_rm_en['data'].apply(strip_multiple_whitespaces)

data_rm_en

,data,player,language,publishedAt
0,Football nFlorian Wirtz s goal Bayer Leverkuse...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,xaSoccer Football Europa League Play Off Seco...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,By Will Pickworth For Mailonline n n n nView ...,Exequiel Palacios,en,2023-02-23T20:53:59Z
3,nBUENOS AIRES AP — World Cup winner Argentina...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,Sign In nSign In nThe Star Edition nCHANGE LOC...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
297,,Moussa Diaby,en,2023-05-04T06:28:32Z
298,By Oliver Salt For Mailonline n n n nView com...,Mykhaylo Mudryk,en,2023-05-05T14:58:51Z
299,,Piero Hincapié,en,2023-04-27T04:57:02Z
300,Man City’s Alex Robertson makes debut Aiden O’...,Piero Hincapié,en,2023-03-24T15:24:08Z


# Put in lower case

# Find sentences with Playername

In [ ]:
data_with_playernames_en = find_lines_with_player(data_wo_pattern_en, data_wo_pattern['player'].unique())

In [ ]:
data_with_playernames

In [ ]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique(), n_lines=10)

In [ ]:
data_with_playernames['data'].iloc[0]